In [1]:
from sqlalchemy import text
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///db_1.db")

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

[('hello world',)]


Выше мы выполняем два SQL-оператора: оператор «CREATE TABLE» [ 1 ] и параметризованный оператор «INSERT» (синтаксис параметризации мы обсудим позже в разделе «Отправка нескольких параметров »). Чтобы зафиксировать работу, проделанную в нашем блоке, мы вызываем Connection.commit()метод, который фиксирует транзакцию. После этого мы можем продолжить выполнение других SQL-операторов и Connection.commit() снова вызывать их для этих операторов. В SQLAlchemy такой стиль называется « commit as you go» (коммит по мере выполнения ).

In [6]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table2 (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table2 (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

Существует также другой стиль фиксации данных. Мы можем заранее объявить наш блок «connect» блоком транзакции. Для этого мы используем метод Engine.begin()для получения соединения, а не Engine.connect()метод . Этот метод будет управлять областью действия Connectionи включать всё внутри транзакции либо с помощью COMMIT в конце, если блок выполнен успешно, либо с помощью ROLLBACK, если возникло исключение. Этот стиль известен как begin once

In [4]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table1 (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

In [11]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

    result = conn.execute(text("select x, y from some_table"))

    for dict_row in result.mappings():
        x = dict_row["x"]
        y = dict_row["y"]

x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
x: 1  y: 1
x: 2  y: 4


In [12]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()

In [14]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x} y: {row.y}")

x: 6 y: 8
x: 9 y: 10
x: 11 y: 12
x: 13 y: 14


Метаданные в базах данных — это данные о данных, которые описывают структуру, содержание, контекст и характеристики самих данных, делая их понятными, управляемыми и доступными, подобно этикеткам на коробке, объясняющим, что внутри. Они включают технические детали (типы данных, названия таблиц, связи) и описательную информацию (автор, дата создания, теги), выступая как «архитектурный план», который организует информацию и придает ей смысл. 

Примеры метаданных в БД:
- Описательные: Названия столбцов ("Фамилия", "Дата рождения"), их описания, ключевые слова, теги.
- Структурные: Информация о таблицах (их названия, количество столбцов) и их взаимосвязях.
- Административные: Дата создания/изменения, права доступа, владелец, история версий.
- Технические: Типы данных (текст, число, дата), форматы хранения, единицы измерения. 
##### Зачем они нужны?
- Поиск и понимание: Помогают находить нужные данные и понимать, что они означают.
- Управление: Обеспечивают управление жизненным циклом данных и контроль доступа.
- Структурирование: Создают «каркас знаний», организуя хаос информации в осмысленную систему.
- Построение интерфейсов: Используются для создания пользовательских форм и отчетов. 

In [18]:
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import MetaData

metadata_obj = MetaData()
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

When do I make a MetaData object in my program?

Having a single MetaData object for an entire application is the most common case, represented as a module-level variable in a single place in an application, often in a “models” or “dbschema” type of package. It is also very common that the MetaData is accessed via an ORM-centric registry or Declarative Base base class, so that this same MetaData is shared among ORM- and Core-declared Table objects.

There can be multiple MetaData collections as well; Table objects can refer to Table objects in other collections without restrictions. However, for groups of Table objects that are related to each other, it is in practice much more straightforward to have them set up within a single MetaData collection, both from the perspective of declaring them, as well as from the perspective of DDL (i.e. CREATE and DROP) statements being emitted in the correct order.

In [19]:
user_table.c.name # .c это column

user_table.c.keys()

['id', 'name', 'fullname']

The first Column in the example user_table includes the Column.primary_key parameter which is a shorthand technique of indicating that this Column should be part of the primary key for this table. The primary key itself is normally declared implicitly and is represented by the PrimaryKeyConstraint construct, which we can see on the Table.primary_key attribute on the Table object:

In [20]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [21]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

In [22]:
metadata_obj.create_all(engine)

In [23]:
BEGIN (implicit)
PRAGMA main.table_...info("user_account")
...
PRAGMA main.table_...info("address")
...
CREATE TABLE user_account (
    id INTEGER NOT NULL,
    name VARCHAR(30),
    fullname VARCHAR,
    PRIMARY KEY (id)
)
...
CREATE TABLE address (
    id INTEGER NOT NULL,
    user_id INTEGER NOT NULL,
    email_address VARCHAR NOT NULL,
    PRIMARY KEY (id),
    FOREIGN KEY(user_id) REFERENCES user_account (id)
)
...
COMMIT

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2318089474.py, line 2)

## Declaring Mapped Classes

##### Establishing a Declarative Base
When using the ORM, the MetaData collection remains present, however it itself is associated with an ORM-only construct commonly referred towards as the Declarative Base. The most expedient way to acquire a new Declarative Base is to create a new class that subclasses the SQLAlchemy DeclarativeBase class:

In [24]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [25]:
sandy = User(name="sandy", fullname="Sandy Cheeks")